In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\riswa\\Desktop\\chicken-disease-prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path    

In [5]:
from src.CNN_Classifier.constants import  CONFIG_FILE_PATH,PARAMS_FILE_PATH
from src.CNN_Classifier.utils.common import read_yaml,create_directories


# UPdating the entity

In [6]:
from src.CNN_Classifier import logger

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path =Path(CONFIG_FILE_PATH),
                  params_file_path = Path(PARAMS_FILE_PATH)):
        self.config_file_path = config_file_path
        self.params_file_path = params_file_path
        logger.info(f"Reading configuration file from: {self.config_file_path}")
        self.config = read_yaml(self.config_file_path)
        logger.info(f"Reading parameters file from: {self.params_file_path}")
        self.params = read_yaml(self.params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config

# updating the componentns

import os


In [8]:
import os
import urllib.request as request
import zipfile
from src.CNN_Classifier import logger
from src.CNN_Classifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_files(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers =request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file)
            logger.info(f"{filename} downloaded info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        logger.info("the data.zip have been unziped sucessfully")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# CRAEATIG PIPELINE

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_files()
    data_ingestion.extract_zip_file()

except Exception as e:
    logger.error(f"Error in data ingestion pipeline: {e}")
    raise

[2024-07-08 04:53:57,007: INFO: 1038593430: Reading configuration file from: config\config.yaml]
[2024-07-08 04:53:57,013: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-08 04:53:57,015: INFO: 1038593430: Reading parameters file from: params.yaml]
[2024-07-08 04:53:57,019: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-08 04:53:57,022: INFO: common: created directory at: artifacts]
[2024-07-08 04:53:57,025: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-08 04:54:03,896: INFO: 1442924126: artifacts/data_ingestion/data.zip downloaded info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X